# 1. Energy data

# M = 30

## 1.0 Startup

In [1]:
disp ('executing gpml startup script...')
mydir = fileparts (mfilename ('fullpath'));                 % where am I located
addpath (mydir)
dirs = {'cov','doc','inf','lik','mean','prior','util'};           % core folders
for d = dirs, addpath (fullfile (mydir, d{1})), end
dirs = {{'util','minfunc'},{'util','minfunc','compiled'}};     % minfunc folders
for d = dirs, addpath (fullfile (mydir, d{1}{:})), end
% addpath([mydir,'/util/sparseinv'])
pkg load statistics

executing gpml startup script...


In [2]:
dataset = 'energy_data'
load ('./data/energy_data.mat');
[ndata, ~] = size(M);
X = M(:,[1:25,27:28]);
Y = M(:,26);
R = randperm(ndata);        
xt = X(R(1:1735),:);
yt = Y(R(1:1735));
R(1:1735) = [];
x = X(R,:);          
y = Y(R);
n = 18000; nt = 1735;   %Ms = 15

dataset = energy_data


In [3]:
sf2 = 1 ; ell = 1 ; sn2 = 0.1 ; 
d = size(x,2);
hyp.cov = log([ones(d,1)*ell;sf2]); hyp.lik = log(sn2); hyp.mean = [];
opts.Xnorm = 'Y' ; opts.Ynorm = 'Y' ;
opts.Ms = 30 ;
opts.ell = ell ; opts.sf2 = sf2 ; opts.sn2 = sn2 ;
opts.meanfunc = []; opts.covfunc = @covSEard; opts.likfunc = @likGauss; opts.inffunc = @infGaussLik ;

partitionCriterion = 'kmeans'; %, 'kmeans', 'knkmeans'
opts.numOptFC = 25 ;
opts.partitionCriterion = partitionCriterion ;
[models,t_dGP_train] = aggregation_train(x,y,opts) ;
save('./res/energymodel30', 'models','t_dGP_train')

In [4]:
partitionCriterion = 'kmeans'; %, 'kmeans', 'knkmeans'
opts.numOptFC = 25 ;
opts.partitionCriterion = partitionCriterion ;
s = load('./res/energymodel30');
models = s.models;


## 1.1 RBCM vs TERBCM

### 1.1.1 RBCM baseline

In [5]:
criterions = {'RBCM'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing energy_data kmeans RBCM
RBCM: SMSE 0.0010, MSLL -0.3105, NLPD 2.5549


### 1.1.2 Grid q

In [6]:
partitionCriterions = {'kmeans'}; % 'random', 'kmeans', 'knkmeans'
criterions = {'TERBCM'}; %, 'TEGRBCM' 'TEGPoE'};
qs = 0.2:0.2:3.0;

for i = 1:length(partitionCriterions)
    partitionCriterion = partitionCriterions{i};
    for j = 1:length(criterions)     
        criterion = criterions{j};
        printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
        opts.partitionCriterion = partitionCriterion ;
        %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
        for q = qs
            [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
            [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
            printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f - q %6.2f\r\n', criterion, SMSE,MSLL,NLPD,q);
            filename = sprintf('./res/terbcm%s_%s_%s_%6.2f.mat', dataset, partitionCriterion, criterion, q);
            save(filename, 'SMSE','MSLL','NLPD')
        endfor
    endfor
endfor


Processing energy_data kmeans TERBCM
TERBCM: SMSE 0.0012, MSLL 0.3891, NLPD 3.2545 - q   0.20
TERBCM: SMSE 0.0011, MSLL -0.9986, NLPD 1.8668 - q   0.40
TERBCM: SMSE 0.0011, MSLL -1.2810, NLPD 1.5844 - q   0.60
TERBCM: SMSE 0.0011, MSLL -1.0448, NLPD 1.8206 - q   0.80
TERBCM: SMSE 0.0010, MSLL -0.3105, NLPD 2.5549 - q   1.00
TERBCM: SMSE 0.0010, MSLL 1.1142, NLPD 3.9796 - q   1.20
TERBCM: SMSE 0.0010, MSLL 3.6541, NLPD 6.5195 - q   1.40
TERBCM: SMSE 0.0009, MSLL 8.1125, NLPD 10.9779 - q   1.60
TERBCM: SMSE 0.0009, MSLL 15.9769, NLPD 18.8423 - q   1.80
TERBCM: SMSE 0.0009, MSLL 30.0187, NLPD 32.8841 - q   2.00
TERBCM: SMSE 0.0008, MSLL 55.4596, NLPD 58.3250 - q   2.20
TERBCM: SMSE 0.0008, MSLL 102.2576, NLPD 105.1230 - q   2.40
TERBCM: SMSE 0.0008, MSLL 189.6163, NLPD 192.4817 - q   2.60
TERBCM: SMSE 0.0008, MSLL 354.9459, NLPD 357.8113 - q   2.80
TERBCM: SMSE 0.0008, MSLL 671.7900, NLPD 674.6554 - q   3.00


### 1.1.3 Find single q

In [7]:
printf('Optimizing q\r\n');
q = grad_q(models, criterion, opts, iter=50, bs=100, lr=0.0001, lambda=10, init_q=1.5);

Optimizing q
Optimizing q: remain iter - 50, q - 1.5000
grad_q_norm = -7846.5
grad_q_reg = -482.27
Optimizing q: remain iter - 49, q - 1.4996
grad_q_norm =   -2.1961e+05
grad_q_reg = -898.15
Optimizing q: remain iter - 48, q - 1.4917
grad_q_norm =   -1.5245e+05
grad_q_reg = -680.50
Optimizing q: remain iter - 47, q - 1.4862
grad_q_norm =   -1.3420e+04
grad_q_reg = -775.28
Optimizing q: remain iter - 46, q - 1.4855
grad_q_norm =   -1.4076e+06
grad_q_reg = -572.13
Optimizing q: remain iter - 45, q - 1.4367
grad_q_norm =   -1.8971e+05
grad_q_reg = -382.36
Optimizing q: remain iter - 44, q - 1.4301
grad_q_norm =   -1.9365e+04
grad_q_reg = -418.59
Optimizing q: remain iter - 43, q - 1.4292
grad_q_norm =   -5.5104e+05
grad_q_reg = -270.20
Optimizing q: remain iter - 42, q - 1.4101
grad_q_norm =   -3.3066e+05
grad_q_reg = -646.71
Optimizing q: remain iter - 41, q - 1.3985
grad_q_norm = -7261.0
grad_q_reg = -297.62
Optimizing q: remain iter - 40, q - 1.3982
grad_q_norm =   -1.2255e+05
grad_q_r

In [8]:
[mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

### 1.1.4 Find multi q

In [9]:
printf('Optimizing q\r\n');
iqs = 1.01 + rand(opts.Ms,1);
qs = mul_grad_q(models, criterion, opts, iter=50, bs=100, lr=0.001, lambda=10, init_q=iqs);
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);

Optimizing q
Optimizing qs: remain iter - 50
qs =

   1.0813
   1.3065
   1.5183
   1.8501
   1.7794
   1.8058
   2.0094
   1.1787
   1.3560
   1.4803
   1.9635
   2.0042
   1.1471
   1.8412
   1.1488
   1.7714
   1.1792
   1.3251
   1.8055
   1.0886
   1.4080
   1.5571
   1.6610
   1.6107
   1.5369
   1.4198
   1.3560
   1.2156
   1.2834
   1.4516

ttb =    2.4277e+05
Optimizing qs: remain iter - 49
qs =

   1.0815
   1.3065
   1.5183
   1.8501
   1.7756
   1.8058
   2.0094
   1.1787
   1.3560
   1.4803
   1.9635
   2.0042
   1.1471
   1.8411
   1.1487
   1.7714
   1.1792
   1.3251
   1.8055
   1.0885
   1.4079
   1.5571
   1.6610
   1.6106
   1.5369
   1.4198
   1.3560
   1.2156
   1.2833
   1.4512

ttb =    1.3879e+06
Optimizing qs: remain iter - 48
qs =

   1.0814
   1.3027
   1.5183
   1.8436
   1.7756
   1.8001
   2.0072
   1.1787
   1.3560
   1.4794
   1.9635
   2.0041
   1.1375
   1.8411
   1.1422
   1.7714
   1.1785
   1.3251
   1.8053
   1.0885
   1.4079
   1.5566
   1.6606
 

   1.42447
   1.32057
   1.34159
   1.21402
   1.28253
   1.44413

ttb =    8.8367e+05
Optimizing qs: remain iter - 28
qs =

   1.07863
   1.29093
   1.51612
   1.43287
   1.19353
   1.79174
   1.93448
   1.07385
   1.35380
   1.46456
   1.75737
   1.25847
   0.75468
   1.79794
   1.02798
   1.77141
   1.15913
   1.20940
   0.93740
   1.08645
   1.40374
   1.38248
   1.61318
   1.60939
   1.42353
   1.32058
   1.34133
   1.21402
   1.27980
   1.44373

ttb =    2.3488e+05
Optimizing qs: remain iter - 27
qs =

   1.07854
   1.29093
   1.51612
   1.43288
   1.19354
   1.79174
   1.93448
   1.07386
   1.35370
   1.46456
   1.75737
   1.25847
   0.75468
   1.79760
   1.02798
   1.77142
   1.15913
   1.20900
   0.93742
   1.08647
   1.40374
   1.38248
   1.61318
   1.60939
   1.42353
   1.32058
   1.34134
   1.21402
   1.27980
   1.44372

ttb =    6.9751e+05
Optimizing qs: remain iter - 26
qs =

   1.07822
   1.29094
   1.51612
   1.43032
   1.19354
   1.79156
   1.90822
   1.07285
   1.3537

   1.34102
   1.21070
   1.27961
   1.44347

ttb =    4.7876e+05
Optimizing qs: remain iter - 7
qs =

   1.07674
   1.28736
   1.51517
   1.40295
   1.18869
   1.77790
   1.87223
   1.03494
   1.35356
   1.45699
   1.74659
   1.25552
   0.75440
   1.77807
   1.02253
   1.77144
   1.15066
   1.19642
   0.92902
   1.08593
   1.40269
   1.33468
   1.60753
   1.58089
   1.39309
   1.29130
   1.34104
   1.21070
   1.27717
   1.44205

ttb =    7.0536e+05
Optimizing qs: remain iter - 6
qs =

   1.07539
   1.28736
   1.51515
   1.40246
   1.18869
   1.77785
   1.87182
   1.03462
   1.35357
   1.45673
   1.74619
   1.25524
   0.75444
   1.77804
   1.02244
   1.77145
   1.15063
   1.19633
   0.92901
   1.08595
   1.40261
   1.33433
   1.60749
   1.58033
   1.39304
   1.28196
   1.34104
   1.21070
   1.27718
   1.44203

ttb =    9.9300e+05
Optimizing qs: remain iter - 5
qs =

   1.07525
   1.28734
   1.51514
   1.40238
   1.18869
   1.77774
   1.87171
   1.03458
   1.35357
   1.45664
   1.74537
 

In [10]:
qs
save('./res/terbcmenergydataqs', 'qs');
printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

qs =

   1.07477
   1.28700
   1.51025
   1.40061
   1.18874
   1.75741
   1.86767
   1.03431
   1.35188
   1.45499
   1.74233
   1.25463
   0.75167
   1.74670
   1.01783
   1.77148
   1.14737
   1.17950
   0.92452
   1.08589
   1.40045
   1.33036
   1.60497
   1.57832
   1.39054
   1.28014
   1.34071
   1.21072
   1.15079
   1.44119

TERBCM: SMSE 0.0010, MSLL 4.2132, NLPD 7.0786


In [11]:
save('./res/terbcmenergydataqs_res', 'SMSE','MSLL','NLPD')

## 1.2 GRBCM vs TEGRBCM

### 1.2.1 GRBCM baseline

In [12]:
criterions = {'GRBCM'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing energy_data kmeans GRBCM
GRBCM: SMSE 0.0011, MSLL -4.7408, NLPD -1.8754


### 1.2.2 Grid q

In [13]:
criterions = {'TEGRBCM'}; %, 'TEGRBCM' 'TEGPoE'};
qs = 0.2:0.2:3.0;

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    for q = qs
        [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
        [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
        printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f - q %6.2f\r\n', criterion, SMSE,MSLL,NLPD,q);
        filename = sprintf('./res/tegrbcm%s_%s_%s_%6.2f.mat', dataset, partitionCriterion, criterion, q);
        save(filename, 'SMSE','MSLL','NLPD')
    endfor
endfor

Processing energy_data kmeans TEGRBCM
TEGRBCM: SMSE 0.0020, MSLL -4.5771, NLPD -1.7117 - q   0.20
TEGRBCM: SMSE 0.0018, MSLL -4.6329, NLPD -1.7675 - q   0.40
TEGRBCM: SMSE 0.0015, MSLL -4.7078, NLPD -1.8424 - q   0.60
TEGRBCM: SMSE 0.0013, MSLL -4.7645, NLPD -1.8991 - q   0.80
TEGRBCM: SMSE 0.0011, MSLL -4.7408, NLPD -1.8754 - q   1.00
TEGRBCM: SMSE 0.0010, MSLL -4.5157, NLPD -1.6503 - q   1.20
TEGRBCM: SMSE 0.0010, MSLL -3.8534, NLPD -0.9880 - q   1.40
TEGRBCM: SMSE 0.0011, MSLL -2.2953, NLPD 0.5701 - q   1.60
TEGRBCM: SMSE 0.0012, MSLL 1.0567, NLPD 3.9221 - q   1.80
TEGRBCM: SMSE 0.0012, MSLL 7.9803, NLPD 10.8457 - q   2.00
TEGRBCM: SMSE 0.0013, MSLL 22.0355, NLPD 24.9009 - q   2.20
TEGRBCM: SMSE 0.0013, MSLL 50.4101, NLPD 53.2755 - q   2.40
TEGRBCM: SMSE 0.0013, MSLL 107.7040, NLPD 110.5694 - q   2.60
TEGRBCM: SMSE 0.0013, MSLL 223.7149, NLPD 226.5803 - q   2.80
TEGRBCM: SMSE 0.0013, MSLL 459.5239, NLPD 462.3893 - q   3.00


### 1.2.3 Find single q

In [14]:
criterion = 'TEGRBCM';

In [15]:
printf('Optimizing q\r\n');
q = grad_q(models, criterion, opts, iter=50, bs=100, lr=0.001, lambda=10, init_q=1.5);

Optimizing q
Optimizing q: remain iter - 50, q - 1.5000
grad_q_norm =   -3.1343e+04
grad_q_reg = -245.38
Optimizing q: remain iter - 49, q - 1.4883
grad_q_norm =   -6.0314e+04
grad_q_reg = -102.17
Optimizing q: remain iter - 48, q - 1.4672
grad_q_norm = -8120.5
grad_q_reg = -158.71
Optimizing q: remain iter - 47, q - 1.4638
grad_q_norm =   -1.8928e+05
grad_q_reg = -192.04
Optimizing q: remain iter - 46, q - 1.3979
grad_q_norm =   -3.0534e+05
grad_q_reg = -113.94
Optimizing q: remain iter - 45, q - 1.2922
grad_q_norm =   -2.6649e+05
grad_q_reg = -81.293
Optimizing q: remain iter - 44, q - 1.2001
grad_q_norm =   -1.6501e+04
grad_q_reg = -20.921
Optimizing q: remain iter - 43, q - 1.1943
grad_q_norm =  981.50
grad_q_reg = -51.093
Optimizing q: remain iter - 42, q - 1.1945
grad_q_norm =   -3.9135e+04
grad_q_reg = -25.948
Optimizing q: remain iter - 41, q - 1.1809
grad_q_norm =   -5.5304e+04
grad_q_reg = -24.590
Optimizing q: remain iter - 40, q - 1.1617
grad_q_norm = -5949.9
grad_q_reg = -

In [16]:
[mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

### 1.2.4 Find multi q

In [17]:
printf('Optimizing q\r\n');
iqs = 1.01 + rand(opts.Ms,1);
qs = mul_grad_q(models, criterion, opts, iter=50, bs=100, lr=0.01, lambda=10, init_q=iqs);
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);

Optimizing q
Optimizing qs: remain iter - 50
qs =

   1.2662
   1.4170
   1.9592
   1.2331
   1.4814
   1.2083
   1.6051
   1.7326
   1.5266
   1.6318
   1.9183
   1.2173
   1.2988
   1.4996
   1.2809
   1.5695
   1.4555
   1.8562
   1.5511
   1.0718
   1.9648
   1.6243
   1.7837
   1.2454
   1.8292
   1.9257
   1.2722
   1.7388
   1.1118
   1.2752

ttb =    3.3130e+04
Optimizing qs: remain iter - 49
qs =

   1.2662
   1.4170
   1.9594
   1.2331
   1.4786
   1.2083
   1.6052
   1.7325
   1.5267
   1.6320
   1.9184
   1.2173
   1.2988
   1.4998
   1.2810
   1.5690
   1.4556
   1.8561
   1.5512
   1.0718
   1.9649
   1.6244
   1.7839
   1.2455
   1.8294
   1.9258
   1.2722
   1.7389
   1.1118
   1.2752

ttb =    5.4325e+04
Optimizing qs: remain iter - 48
qs =

   1.2662
   1.4164
   1.9589
   1.1437
   1.4723
   1.2080
   1.6037
   1.5263
   1.5267
   1.6314
   1.9087
   1.2134
   1.2985
   1.4704
   1.2791
   1.5598
   1.4522
   1.5016
   1.5029
   1.0413
   1.9571
   1.6249
   1.7686
 

   0.83605
   0.82851
   1.61591
   1.40300
   1.48274
   1.17817
   1.01284
   1.22487
   1.11325
   1.14233
   1.07990
   0.96559

ttb =    1.1419e+04
Optimizing qs: remain iter - 28
qs =

   1.26619
   1.23624
   1.94205
   0.76065
   1.23820
   1.12403
   1.40141
   0.59438
   1.26389
   1.39219
   1.53102
   1.13230
   1.07180
   1.12242
   0.96917
   1.40255
   1.16734
   0.93497
   0.83292
   0.82408
   1.61723
   1.40374
   1.48136
   1.17696
   1.00914
   1.22322
   1.11325
   1.14254
   1.07876
   0.95525

ttb =  5384.9
Optimizing qs: remain iter - 27
qs =

   1.26619
   1.23627
   1.94215
   0.76064
   1.23778
   1.12405
   1.40146
   0.59449
   1.26398
   1.39227
   1.53101
   1.13229
   1.07181
   1.12249
   0.96917
   1.40229
   1.16738
   0.93495
   0.83291
   0.82408
   1.61729
   1.40380
   1.48148
   1.17699
   1.00915
   1.22322
   1.11322
   1.14256
   1.07876
   0.95525

ttb =    1.0162e+04
Optimizing qs: remain iter - 26
qs =

   1.26619
   1.23618
   1.94011
   0

   0.62261

ttb =  3626.1
Optimizing qs: remain iter - 7
qs =

   1.26619
   1.02741
   1.79861
   0.65264
   1.23548
   1.05344
   1.23374
   0.58299
   1.24339
   1.25444
   1.30964
   1.02500
   1.00746
   0.91225
   0.90871
   1.29797
   0.97940
   0.51336
   0.51550
   0.46514
   1.35118
   1.31795
   1.26783
   1.14709
   0.37502
   0.96500
   0.36369
   1.04570
   0.98674
   0.62248

ttb =  6897.6
Optimizing qs: remain iter - 6
qs =

   1.26619
   1.01707
   1.64923
   0.65248
   1.23609
   1.05273
   1.22631
   0.58293
   1.24380
   1.24964
   1.30864
   1.02500
   1.00024
   0.91214
   0.89996
   1.29824
   0.97587
   0.51371
   0.51615
   0.46522
   1.34698
   1.27562
   1.26001
   1.13780
   0.37932
   0.96204
   0.36386
   1.04440
   0.98641
   0.62248

ttb =  3536.0
Optimizing qs: remain iter - 5
qs =

   1.26619
   1.01649
   1.64970
   0.65239
   1.23622
   1.05279
   1.22686
   0.58288
   1.24324
   1.24925
   1.30523
   1.02370
   1.00015
   0.91202
   0.89936
   1.293

In [18]:
qs
save('./res/tegrbcmenergydataqs', 'qs');
printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

qs =

   1.26619
   1.00773
   1.65389
   0.53619
   1.23579
   1.05129
   1.22478
   0.53161
   1.21447
   1.24832
   1.21898
   0.97853
   0.99488
   0.88199
   0.84631
   1.27726
   0.97545
   0.53070
   0.52375
   0.47783
   1.34454
   1.21718
   1.25925
   1.12096
   0.39497
   0.86128
   0.37009
   1.03401
   0.96757
   0.61763

TEGRBCM: SMSE 0.0013, MSLL -4.5568, NLPD -1.6914


In [19]:
save('./res/tegrbcmenergydataqs_res', 'SMSE','MSLL','NLPD')

## 1.3 GPoE vs TEGPoE

### 1.3.1 GPoE baseline

In [20]:
criterions = {'GPoE'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing energy_data kmeans GPoE
GPoE: SMSE 0.0011, MSLL -0.1847, NLPD 2.6807


### 1.3.2 Grid q

In [21]:
partitionCriterions = {'kmeans'}; % 'random', 'kmeans', 'knkmeans'
criterions = {'TEGPoE'}; %, 'TEGRBCM' 'TEGPoE'};
qs = 0.2:0.2:3.0;

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    for q = qs
        [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
        [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
        printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f - q %6.2f\r\n', criterion, SMSE,MSLL,NLPD,q);
        filename = sprintf('./res/tegpoe%s_%s_%s_%6.2f.mat', dataset, partitionCriterion, criterion, q);
        save(filename, 'SMSE','MSLL','NLPD')
    endfor
endfor

Processing energy_data kmeans TEGPoE
TEGPoE: SMSE 0.0012, MSLL 0.7031, NLPD 3.5685 - q   0.20
TEGPoE: SMSE 0.0012, MSLL -0.7845, NLPD 2.0809 - q   0.40
TEGPoE: SMSE 0.0012, MSLL -1.1117, NLPD 1.7537 - q   0.60
TEGPoE: SMSE 0.0011, MSLL -0.9021, NLPD 1.9633 - q   0.80
TEGPoE: SMSE 0.0011, MSLL -0.1847, NLPD 2.6807 - q   1.00
TEGPoE: SMSE 0.0010, MSLL 1.2302, NLPD 4.0956 - q   1.20
TEGPoE: SMSE 0.0010, MSLL 3.7671, NLPD 6.6325 - q   1.40
TEGPoE: SMSE 0.0009, MSLL 8.2302, NLPD 11.0956 - q   1.60
TEGPoE: SMSE 0.0009, MSLL 16.1100, NLPD 18.9754 - q   1.80
TEGPoE: SMSE 0.0009, MSLL 30.1833, NLPD 33.0487 - q   2.00
TEGPoE: SMSE 0.0008, MSLL 55.6823, NLPD 58.5477 - q   2.20
TEGPoE: SMSE 0.0008, MSLL 102.5847, NLPD 105.4501 - q   2.40
TEGPoE: SMSE 0.0008, MSLL 190.1321, NLPD 192.9975 - q   2.60
TEGPoE: SMSE 0.0008, MSLL 355.8074, NLPD 358.6728 - q   2.80
TEGPoE: SMSE 0.0008, MSLL 673.2961, NLPD 676.1615 - q   3.00


### 1.3.3 Find single q

In [22]:
criterion = 'TEGPoE';

In [23]:
printf('Optimizing q\r\n');
q = grad_q(models, criterion, opts, iter=150, bs=100, lr=0.001, lambda=10, init_q=1.5);

Optimizing q
Optimizing q: remain iter - 150, q - 1.5000
grad_q_norm =   -2.1441e+06
grad_q_reg = -256.48
Optimizing q: remain iter - 149, q - 0.7598
grad_q_norm =   -2.4540e+04
grad_q_reg =  0.60651
Optimizing q: remain iter - 148, q - 0.7513
grad_q_norm = -1077.8
grad_q_reg = -6.7901
Optimizing q: remain iter - 147, q - 0.7509
grad_q_norm =  133.97
grad_q_reg =  2.6439
Optimizing q: remain iter - 146, q - 0.7510
grad_q_norm =   -1.7983e+04
grad_q_reg = -27.207
Optimizing q: remain iter - 145, q - 0.7447
grad_q_norm = -7032.7
grad_q_reg =  1.0378
Optimizing q: remain iter - 144, q - 0.7423
grad_q_norm =   -8.2869e+05
grad_q_reg =  6.4808
Optimizing q: remain iter - 143, q - 0.4565
grad_q_norm =    8.7035e+05
grad_q_reg =  101.72
Optimizing q: remain iter - 142, q - 0.7570
grad_q_norm = -4410.3
grad_q_reg =  5.5937
Optimizing q: remain iter - 141, q - 0.7555
grad_q_norm = -588.63
grad_q_reg = -0.48059
Optimizing q: remain iter - 140, q - 0.7553
grad_q_norm = -8881.7
grad_q_reg = -22.31

grad_q_norm =  8858.5
grad_q_reg =  54.062
Optimizing q: remain iter - 58, q - 0.5687
grad_q_norm =  5898.7
grad_q_reg =  67.168
Optimizing q: remain iter - 57, q - 0.5710
grad_q_norm =   -5.5637e+04
grad_q_reg =  54.302
Optimizing q: remain iter - 56, q - 0.5520
grad_q_norm =  9489.4
grad_q_reg =  75.352
Optimizing q: remain iter - 55, q - 0.5555
grad_q_norm =  1659.1
grad_q_reg =  57.402
Optimizing q: remain iter - 54, q - 0.5563
grad_q_norm =  9608.6
grad_q_reg =  68.142
Optimizing q: remain iter - 53, q - 0.5598
grad_q_norm =    2.7761e+04
grad_q_reg =  66.671
Optimizing q: remain iter - 52, q - 0.5696
grad_q_norm =  3188.7
grad_q_reg =  53.314
Optimizing q: remain iter - 51, q - 0.5709
grad_q_norm =  4526.0
grad_q_reg =  66.142
Optimizing q: remain iter - 50, q - 0.5727
grad_q_norm =  7963.0
grad_q_reg =  43.539
Optimizing q: remain iter - 49, q - 0.5756
grad_q_norm =  4194.5
grad_q_reg =  40.877
Optimizing q: remain iter - 48, q - 0.5772
grad_q_norm =  4689.1
grad_q_reg =  58.305

In [24]:
[mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

### 1.3.4 Find multi q

In [25]:
printf('Optimizing q\r\n');
iqs = 1.01 + rand(opts.Ms,1);
qs = mul_grad_q(models, criterion, opts, iter=50, bs=100, lr=0.1, lambda=10, init_q=iqs);
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);

Optimizing q
Optimizing qs: remain iter - 50
qs =

   2.0021
   1.0688
   1.6409
   1.8042
   1.5155
   1.2849
   1.0346
   1.0125
   1.2837
   1.1247
   1.6984
   1.4582
   1.5109
   1.3247
   1.9463
   1.4898
   1.8479
   1.4020
   1.0323
   1.3669
   1.2513
   1.1138
   1.3917
   1.1065
   1.4631
   1.6601
   1.2262
   1.3465
   1.5778
   1.9209

ttb = 0
Optimizing qs: remain iter - 49
qs =

   1.6685
   1.0695
   1.6250
   1.7988
   1.5158
   1.2851
   1.0350
   1.0121
   1.2844
   1.1249
   1.6984
   1.3512
   1.5118
   1.3254
   1.9457
   1.4917
   1.8476
   1.4030
   1.0341
   1.3672
   1.2452
   1.1144
   1.3921
   1.1071
   1.4643
   1.5916
   1.2252
   1.3470
   1.5782
   1.9176

ttb = 0
Optimizing qs: remain iter - 48
qs =

   1.64407
   1.06976
   1.62513
   1.79971
   1.51062
   1.28526
   1.03626
   1.01929
   1.27761
   1.12530
   1.69923
   1.35273
   1.51220
   1.04681
   1.94616
   1.49439
   1.84795
   0.60300
   1.03326
   1.36919
   1.24566
   1.11456
   1.39246
  

   0.79709
   0.96396
   0.61669
   1.21138
   1.33815

ttb = 0
Optimizing qs: remain iter - 28
qs =

   0.83257
   0.45788
   1.59451
   0.76630
   0.53579
   0.57808
   0.81479
   0.77573
   1.10455
   0.53147
   1.20864
   1.05304
   0.68413
   0.74680
   0.88966
   1.45890
   0.81350
   0.63915
   0.65453
   1.19317
   0.97573
   0.61739
   0.98795
   0.78410
   0.62797
   0.79776
   0.96597
   0.61675
   1.21138
   1.33871

ttb = 0
Optimizing qs: remain iter - 27
qs =

   0.825758
   0.460185
   1.592848
  -0.033705
   0.539173
   0.578079
   0.814252
  -0.024270
   1.118890
   0.534117
   1.088652
   0.960368
   0.682777
   0.798645
   0.686120
   1.469503
   0.798219
   1.052941
   0.658512
   0.868599
   0.970335
   0.644035
   0.986398
   0.784593
   0.630928
   0.791005
   0.963741
   0.619674
   1.221073
   1.292188

ttb = 0
Optimizing qs: remain iter - 26
qs =

   0.819456
   0.463250
   1.536306
   0.766295
   0.539585
   0.579473
   0.814629
   0.775730
   1.119862
   0.5

Optimizing qs: remain iter - 9
qs =

   0.8160070
  -8.3195368
  -8.4035696
  -7.2991601
  -9.0590088
  -8.6909371
   0.0162974
  -7.4086245
  -8.4779987
  -9.8501696
  -8.7293640
  -7.2293245
  -9.2322151
  -0.8056901
  -0.0136020
  -8.6312328
   0.0021152
  -8.5332674
  -8.8534492
  -7.1008419
  -9.4285072
  -4.8406438
  -9.4115282
  -9.2813089
  -8.8473259
   0.7995018
  -8.6234004
  -8.7548803
  -8.3680429
  -8.3550395

ttb = 0
Optimizing qs: remain iter - 8
qs =

   1.6007e-02
  -9.1195e+00
  -9.2036e+00
  -8.0992e+00
  -9.8590e+00
  -9.4909e+00
   8.1630e-01
  -8.2086e+00
  -9.2780e+00
  -1.0650e+01
  -9.5294e+00
  -8.0293e+00
  -1.0032e+01
  -1.6057e+00
   7.8640e-01
  -9.4312e+00
   8.0212e-01
  -9.3333e+00
  -9.6534e+00
  -7.9008e+00
  -1.0229e+01
  -5.6406e+00
  -1.0212e+01
  -1.0081e+01
  -9.6473e+00
  -4.9821e-04
  -9.4234e+00
  -9.5549e+00
  -9.1680e+00
  -9.1550e+00

ttb = 0
Optimizing qs: remain iter - 7
qs =

   8.1601e-01
  -9.9195e+00
  -1.0004e+01
  -8.8992e+00
  -1.

In [26]:
qs
save('./res/tegpoeenergydataqs', 'qs');
printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

qs =

   1.6007e-02
  -1.2320e+01
  -1.2404e+01
  -1.1299e+01
  -1.3059e+01
  -1.2691e+01
   8.1630e-01
  -1.1409e+01
  -1.2478e+01
  -1.2250e+01
  -1.2729e+01
  -1.2829e+01
  -1.3232e+01
  -8.0057e+00
   7.8640e-01
  -1.2631e+01
  -3.9979e+00
  -1.2533e+01
  -1.1253e+01
  -1.2701e+01
  -1.1829e+01
  -1.2041e+01
  -1.1812e+01
  -1.3281e+01
  -1.1247e+01
  -4.9821e-04
  -1.4223e+01
  -1.2755e+01
  -1.2368e+01
  -1.2355e+01

TEGPoE: SMSE 0.0015, MSLL -5.8110, NLPD -2.9456


In [27]:
save('./res/tegpoeenergydataqs_res', 'SMSE','MSLL','NLPD')